# Yield Management 


## Objective and Prerequisites

See how mathematical optimization can make your revenues and profits soar in this example, where we’ll show you how an airline can use the AI technology to devise an optimal seat pricing strategy. You’ll learn how to formulate this Yield Management Problem as a three-period stochastic programming problem using the Gurobi Python API and solve it with the Gurobi Optimizer.

This model is example 24 from the fifth edition of Model Building in Mathematical Programming by H. Paul Williams on pages 282-284 and 337-340.

This modeling example is at the advanced level, where we assume that you know Python and the Gurobi Python API and that you have advanced knowledge of building mathematical optimization models. Typically, the objective function and/or constraints of these examples are complex or require advanced features of the Gurobi Python API.

**Download the Repository** <br /> 
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). 

**Gurobi License** <br /> 
In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an [evaluation license](https://www.gurobi.com/downloads/request-an-evaluation-license/) as a *commercial user*, or download a [free license](https://www.gurobi.com/academia/academic-program-and-licenses/) as an *academic user*.

## Problem Description

An airline is selling tickets for flights to a particular destination. The flights will depart in three weeks and require up to six planes, each costing £50 000 to hire. Each plane has the following:

* 37 First Class seats
* 38 Business Class seats
* 47 Economy Class seats.

The airline needs to decide on an initial price for each of these seats, and will then have the opportunity to update the price after one week and two weeks. Once a customer has purchased a ticket, there is no cancellation option. For administrative simplicity, three price level options are possible in each class (one of which must be chosen). The same option need not be chosen for each class. These are given in the following table for the current period (period 1) and two future periods.

![optionsClass](optionsClass.PNG)

Demand is uncertain but will be affected by price. Demand forecasts have been made according to a probability distribution that divides the demand levels into three scenarios for each period. The probabilities of the three scenarios
in each period are as follows:

![scenariosProb](scenariosProb.PNG)

The forecasted demand levels are shown in the following table:

![forecastDemand](forecastDemand.PNG)

The goal is to determine the price levels for the current period, how many seats to sell in each class, the provisional number of planes to book and provisional price levels and seats to sell in future periods in order to maximize
expected yield. We should be able to meet commitments under all possible combinations of scenarios. With hindsight (i.e. not known until the beginning of the next period), it turned out that demand in each period (depending on the price level you chose) was as shown in following table.

![actualDemand](actualDemand.PNG)

We use the actual demand that resulted from the prices we set in period 1 to rerun the model at the beginning of period 2 to set price levels for period 2 and provisional price levels for period 3. We repeat this procedure with a rerun at the beginning of period 3.

---
## Model Formulation

The Yield Management problem is formulated as a three-period stochastic programming problem. Solving this model for the first time gives recommendations for price levels and sales of week 1 and recommends price levels and sales for subsequent weeks under all possible scenarios. The probabilities of these scenarios will be taken into account in order to maximize expected yield. A week later the model will be rerun, taking into account the committed sales and revenue in the first week, to redetermine recommended prices and sales for the second week (i.e. with ‘recourse’) and the third week under all possible scenarios. The procedure will be repeated again a week later.

### Sets and indices

$i,j,k \in \text{Scenarios}$: Indices and set of scenarios.

$h \in \text{Options}$: Index and set of price options.

$c \in \text{Class}$: Index and set of seats categories.

### Parameters

$\text{price1}_{c,h} \in \mathbb{R}^+$: Price of option $h$ chosen for class $c$ in week 1.

$\text{price2}_{i,c,h} \in \mathbb{R}^+$: Price of option $h$ chosen for class $c$ in week 2 as a result of scenario $i$ in week 1.

$\text{price3}_{i,j,c,h} \in \mathbb{R}^+$: Price of option $h$ chosen for class $c$ in week 3 as a result of scenario $i$ in week 1, and scenario $j$ in week 2.

$\text{forecast1}_{i,c,h} \in \mathbb{R}^+$: Forecast demand in week 1 for class $c$ under price option $h$ and scenario $i$.

$\text{forecast2}_{i,j,c,h} \in \mathbb{R}^+$: Forecast demand in week 2 for class $c$ under price option $h$ if scenario $i$ holds in week 1, and scenario $j$ in week 2.

$\text{forecast3}_{i,j,k,c,h} \in \mathbb{R}^+$: Forecast demand in week 3 for class $c$ under price option $h$ if  scenario $i$ holds in week 1, scenario $j$ in week 2, and scenario $k$ in week 3.

$\text{prob}_i \in [0,1]$: Probability of scenario $i$.

$\text{cap}_c \in \mathbb{N}$: Capacity per plane for class $c$.

$\text{cost} \in \mathbb{R}^+$: Cost to hire a plane.


### Decision Variables

$p1_{c,h} \in \{0, 1\}$: This binary variable is equal to one if price of option $h$ is chosen for class $c$ in week 1.

$p2_{i,c,h} \in \{0, 1\}$: This binary variable is equal to one if price of option $h$ is chosen for class $c$ in week 2 as a result of scenario $i$ in week 1.

$p3_{i,j,c,h} \in \{0, 1\}$: This binary variable is equal to one if price of option $h$ is chosen for class $c$ in week 3 as a result of scenario $i$ in week 1 and scenario $j$ in week 2.

$s1_{i,c,h} \in \mathbb{R}^+$: Number of tickets to be sold in week 1 for class $c$ under price option $h$ and scenario $i$.

$s2_{i,j,c,h} \in \mathbb{R}^+$: Number of tickets to be sold in week 2 for class $c$ under price option $h$ if scenario $i$ holds in week 1, and scenario $j$ in week 2.

$s3_{i,j,k,c,h} \in \mathbb{R}^+$: Number of tickets to be sold in week 3 for class $c$ under price option $h$ if  scenario $i$ in week 1, scenario $j$ in week 2, and scenario $k$ in week 3.

$n \in \mathbb{N}$: Number of planes to fly.

### Objective function

**Profit**: Maximize expected profit.

$$
\text{Maximize} \quad \text{profit} = (
\sum_{i \in \text{Scenarios}} \sum_{c \in \text{Class}} 
\sum_{h \in \text{Options}}{\text{prob}_i * \text{price1}_{c,h} * p1_{c,h} * s1_{i,c,h}  }  +
$$

$$
\sum_{i \in \text{Scenarios}} \sum_{j \in \text{Scenarios}} \sum_{c \in \text{Class}} \sum_{h \in \text{Options}}
{\text{prob}_i *\text{prob}_j * \text{price2}_{c,h} * p2_{i,c,h} * s2_{i,j,c,h} } +
$$

$$
\sum_{i \in \text{Scenarios}} \sum_{j \in \text{Scenarios}} \sum_{k \in \text{Scenarios}} 
\sum_{c \in \text{Class}} \sum_{h \in \text{Options}}
{\text{prob}_i * \text{prob}_j * \text{prob}_k * \text{price3}_{c,h} * p2_{i,j,c,h} * s3_{i,j,k,c,h}} )
- \text{cost} * n
$$

### Constraints

**Price option week 1**: Only one price option must be chosen in each class in week 1.

$$
\sum_{h \in \text{Options}} p1_{c,h} = 1 \quad \forall c \in \text{Class}
$$

**Sales week 1**: Sales cannot exceed forecasted demand in week 1.

$$
s1_{i,c,h} \leq \text{forecast1}_{i,c,h} * p1_{c,h},
\quad \forall i \in \text{Scenarios}, \; c \in \text{Class}, \; h \in \text{Options}
$$

**Price option week 2**: Only one price option must be chosen in each class in week 2 for each scenario in week 1.

$$
\sum_{h \in \text{Options}} p2_{i,c,h} = 1 \quad \forall c \in \text{Class}, \; i \in \text{Scenarios}
$$

**Sales week 2**: Sales cannot exceed forecasted demand in week 2.

$$
s2_{i,j,c,h} \leq \text{forecast2}_{j,c,h} * p2_{i,c,h},
\quad \forall i,j \in \text{Scenarios}, \; c \in \text{Class}, \; h \in \text{Options}
$$

**Price option week 3**: Only one price option must be chosen in each class in week 3 for each scenario in week 1 and week 2.

$$
\sum_{h \in \text{Options}} p3_{i,j,c,h} = 1 \quad \forall c \in \text{Class}, \; i,j \in \text{Scenarios}
$$

**Sales week 3**: Sales cannot exceed forecasted demand in week 3.
$$
s3_{i,j,k,c,h} \leq \text{forecast3}_{k,c,h} * p3_{i,j,c,h},
\quad \forall i,j,k \in \text{Scenarios}, \; c \in \text{Class}, \; h \in \text{Options}
$$

**Class capacity**: Capacity constraint for each class.

$$
\sum_{h \in \text{Options}} s1_{i,c,h} +
\sum_{h \in \text{Options}} s2_{i,j,c,h} + 
\sum_{h \in \text{Options}} s3_{i,j,k,c,h} \leq \text{cap}_c * n
\quad \forall i,j,k \in \text{Scenarios}, \; c \in \text{Class}
$$

**Planes** Up to six planes can be hired.

$$
n \leq 6
$$

---
## Python Implementation

We import the Gurobi Python Module and other Python libraries.

In [ ]:
%pip install gurobipy
# Note, the restricted license is not sufficient to run this notebook, a full license is needed

In [1]:
import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.1.0

## Input Data
We define all the input data for the model.

In [2]:
# Lists of classes, price options, and scenarios.

classes = ['First', 'Business', 'Economy']

options = ['option1', 'option2', 'option3']

scenarios = ['sce1', 'sce2', 'sce3']

#  Classes, price options, and prices value for week 1, 2, and 3.

ch, price1, price2, price3  = gp.multidict({
    ('First', 'option1'): [1200, 1400, 1500],
    ('Business', 'option1'): [900, 1100, 820],
    ('Economy', 'option1'): [500, 700, 480],
    ('First', 'option2'): [1000, 1300, 900],
    ('Business', 'option2'): [800, 900, 800],
    ('Economy', 'option2'): [300, 400, 470],
    ('First', 'option3'): [950, 1150, 850],
    ('Business', 'option3'): [600, 750, 500],
    ('Economy', 'option3'): [200, 350, 450]
})

# Probablity of each scenario

prob ={'sce1': 0.1, 'sce2': 0.7, 'sce3': 0.2}

# Forecasted demand for each class, price option, and scenario at week 1

ich, fcst1, fcst2, fcst3 = gp.multidict({
    ('sce1', 'First', 'option1'): [10, 20, 30],
    ('sce1', 'Business', 'option1'): [20, 42, 40],
    ('sce1', 'Economy', 'option1'): [45, 50, 50],
    ('sce1', 'First', 'option2'): [15, 25, 35],
    ('sce1', 'Business', 'option2'): [25, 45, 50],
    ('sce1', 'Economy', 'option2'): [55, 52, 60],
    ('sce1', 'First', 'option3'): [20, 35, 40],
    ('sce1', 'Business', 'option3'): [35, 46, 55],
    ('sce1', 'Economy', 'option3'): [60, 60, 80],
    ('sce2', 'First', 'option1'): [20, 10, 30],
    ('sce2', 'Business', 'option1'): [40, 50, 10],
    ('sce2', 'Economy', 'option1'): [50, 60, 50],
    ('sce2', 'First', 'option2'): [25, 40, 40],
    ('sce2', 'Business', 'option2'): [42, 60, 40],
    ('sce2', 'Economy', 'option2'): [52, 65, 60],
    ('sce2', 'First', 'option3'): [35, 50, 60],
    ('sce2', 'Business', 'option3'): [45, 80, 45],
    ('sce2', 'Economy', 'option3'): [63, 90, 70],
    ('sce3', 'First', 'option1'): [45, 50, 50],
    ('sce3', 'Business', 'option1'): [45, 20, 40],
    ('sce3', 'Economy', 'option1'): [55, 10, 60],
    ('sce3', 'First', 'option2'): [50, 55, 70],
    ('sce3', 'Business', 'option2'): [46, 30, 45],
    ('sce3', 'Economy', 'option2'): [56, 40, 65],
    ('sce3', 'First', 'option3'): [60, 80, 80],
    ('sce3', 'Business', 'option3'): [47, 50, 60],
    ('sce3', 'Economy', 'option3'): [64, 60, 70]    
})

#  Actual demand at weeks 1, 2, and 3.

ch, demand1, demand2, demand3  = gp.multidict({
    ('First', 'option1'): [25, 22, 45],
    ('Business', 'option1'): [50, 45, 20],
    ('Economy', 'option1'): [50, 50, 55],
    ('First', 'option2'): [30, 45, 60],
    ('Business', 'option2'): [40, 55, 40],
    ('Economy', 'option2'): [53, 60, 60],
    ('First', 'option3'): [40, 50, 75],
    ('Business', 'option3'): [45, 75, 50],
    ('Economy', 'option3'): [65, 80, 75]
})

# Class capacity

cap ={'First': 37, 'Business': 38, 'Economy': 47}

# cost per plane

cost = 50000



In [3]:
# Preprocessing

# week 1 data structures
list_ch = []

for c in classes:
    for h in options:
        tp = c,h
        list_ch.append(tp)

ch = gp.tuplelist(list_ch)

list_ich = []

for i in scenarios:
    for c in classes:
        for h in options:
            tp = i,c,h
            list_ich.append(tp)

ich = gp.tuplelist(list_ich)

# week 2 data structure

list_ijch = []

for i in scenarios:
    for j in scenarios:
        for c in classes:
            for h in options:
                tp = i,j,c,h
                list_ijch.append(tp)

ijch = gp.tuplelist(list_ijch)

# week 3 data structure

list_ijkch = []

for i in scenarios:
    for j in scenarios:
        for k in scenarios:
            for c in classes:
                for h in options:
                    tp = i,j,k,c,h
                    list_ijkch.append(tp)

ijkch = gp.tuplelist(list_ijkch)

# scenarios data structure

list_ijk = []

for i in scenarios:
    for j in scenarios:
        for k in scenarios:
            tp = i,j,k,
            list_ijk.append(tp)

ijk = gp.tuplelist(list_ijk)

# capacity constraints data structure

list_ijkc = []

for i in scenarios:
    for j in scenarios:
        for k in scenarios:
            for c in classes:
                tp = i,j,k,c
                list_ijkc.append(tp)

ijkc = gp.tuplelist(list_ijkc)

## Model Deployment

Solving bilinear problems with Gurobi is as easy as configuring the global parameter `nonConvex`, and setting this parameter to the value of 2.

### First Period Model

At the beginning of week 1, we want to determine the price options for this week.

We create a model and the variables.

In [4]:
model = gp.Model('YieldManagement')

# Set global parameters. 
model.params.nonConvex = 2

# Decision variables

# price option binary variables at each week
p1ch = model.addVars(ch, vtype=GRB.BINARY, name="p1ch")
p2ich = model.addVars(ich, vtype=GRB.BINARY, name="p2ich")
p3ijch = model.addVars(ijch, vtype=GRB.BINARY, name="p3ijch")

# tickets to be sold at each week
s1ich = model.addVars(ich, name="s1ich")
s2ijch = model.addVars(ijch, name="s2ijch")
s3ijkch = model.addVars(ijkch, name="s3ijkch")

# number of planes to fly
n = model.addVar(ub=6, vtype=GRB.INTEGER, name="planes")

Using license file c:\gurobi\gurobi.lic
Changed value of parameter nonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1


### Week 1 constraints

The following constraints ensure that only one price option is chosen in each class in week 1.

In [5]:
# Price option constraints for week 1

priceOption1 = model.addConstrs( (gp.quicksum(p1ch[c,h] for h in options ) == 1 for c in classes ), name='priceOption1' )

The following constraints enforce that sales cannot exceed forecasted demand in week 1.

In [6]:
# sales constraints for week 1

sales1 = model.addConstrs( (s1ich[i,c,h] <= fcst1[i,c,h]*p1ch[c,h] for i,c,h in ich ), name='sales1' )

### Week 2 constraints

Only one price option must be chosen in each class in week 2 for each scenario in week 1.

In [7]:
# Price option constraints for week 2

priceOption2 = model.addConstrs( (gp.quicksum(p2ich[i,c,h] for h in options ) 
                                  == 1 for i in scenarios for c in classes ), name='priceOption2' )

Sales cannot exceed forecasted demand in week 2.

In [8]:
# sales constraints for week 2

sales2 = model.addConstrs( (s2ijch[i,j,c,h] <= fcst2[j,c,h]*p2ich[i,c,h] for i,j,c,h in ijch ), name='sales2' )

### Week 3 constraints

Only one price option must be chosen in each class in week 3 for each scenario in week 1 and week 2.

In [9]:
# Price option constraints for week 3

priceOption3 = model.addConstrs( (gp.quicksum(p3ijch[i,j,c,h] for h in options ) 
                                  == 1 for i in scenarios for j in scenarios for c in classes ), name='priceOption3' )

Sales cannot exceed forecasted demand in week 3.

In [10]:
# sales constraints for week 3

sales3 = model.addConstrs( (s3ijkch[i,j,k,c,h] <= fcst3[k,c,h]*p3ijch[i,j,c,h] for i,j,k,c,h in ijkch ), name='sales3' )

Capacity constraint for each class.

In [11]:
# Class capacity constraints

classCap = model.addConstrs( (gp.quicksum(s1ich[i,c,h] for h in options)  
                              + gp.quicksum(s2ijch[i,j,c,h] for h in options) 
                              + gp.quicksum(s3ijkch[i,j,k,c,h] for h in options)  
                              <= cap[c]*n for i,j,k,c in ijkc ) , name='classCap')

The objective is to maximize expected profit.

In [12]:
# Objective function
obj = gp.quicksum(prob[i]*price1[c,h]*p1ch[c,h]*s1ich[i,c,h] for i,c,h in ich ) \
+ gp.quicksum(prob[i]*prob[j]*price2[c,h]*p2ich[i,c,h]*s2ijch[i,j,c,h] for i,j,c,h in ijch ) \
+ gp.quicksum(prob[i]*prob[j]*prob[k]*price3[c,h]*p3ijch[i,j,c,h]*s3ijkch[i,j,k,c,h] for i,j,k,c,h in ijkch) - cost*n

model.setObjective( obj, GRB.MAXIMIZE)

In [13]:
# Verify model formulation

model.write('YieldManagement.lp')

# Run optimization engine

model.optimize()

#############################################################
#            Print results of model for week 1
#############################################################

print("\n\n\n____________________Week 1 solution___________________________")

print(f"The expected total profit is: £{round(model.objVal,2): ,}") 
print(f"Number of planes to book: {n.x}")

# Week 1 prices for seat class

# optimal values of option prices at week 1
opt_p1ch = {}

print("\n____________________Week 1 prices_______________________________")
for c,h in ch:
    opt_p1ch[c,h] = 0
    if p1ch[c,h].x > 0.5:
        opt_p1ch[c,h] = round(p1ch[c,h].x)
        price_ch = opt_p1ch[c,h]*price1[c,h]
        print(f"({c},{h}) = £{price_ch: ,}")
#

# Week 2 provisional prices

print("\n_____________Week 2 provisional prices____________________________")
for i,c,h in ich:
    if p2ich[i,c,h].x > 0.5:
        price_ch = round(p2ich[i,c,h].x)*price2[c,h]
        print(f"({i}, {c}, {h}) = £{price_ch: ,}")

# Week 3 provisional prices

print("\n_____________Week 3 provisional prices____________________________")
for i,j,c,h in ijch:
    if p3ijch[i,j,c,h].x > 0.5:
        price_ch = round(p3ijch[i,j,c,h].x)*price3[c,h]
        print(f"({i}, {j}, {c}, {h}) = £{price_ch: ,}")

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 471 rows, 469 columns and 1629 nonzeros
Model fingerprint: 0x21b1ab1b
Model has 351 quadratic objective terms
Variable types: 351 continuous, 118 integer (117 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [5e+04, 5e+04]
  QObjective range [9e-01, 2e+03]
  Bounds range     [1e+00, 6e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -300000.0000
Presolve time: 0.00s
Presolved: 822 rows, 820 columns, 2682 nonzeros
Variable types: 702 continuous, 118 integer (117 binary)

Root relaxation: objective -1.704970e+05, 487 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 170497.004    0   15 -300000.00 170497.004   157%     -    0s
H    0     0

### Second Period model

We ran the model using the actual demand of week 1 and determine the price options for week 2.

In [14]:
model2 = gp.Model('YieldManagement2')

# Set global parameters 
model2.params.nonConvex = 2

# Decision variables

# price option binary variables at each week
p1ch = model2.addVars(ch, vtype=GRB.BINARY, name="p1ch")

# Fix price options of week 1
for c,h in ch:
    p1ch[c,h].lb = opt_p1ch[c,h]

p2ich = model2.addVars(ich, vtype=GRB.BINARY, name="p2ich")
p3ijch = model2.addVars(ijch, vtype=GRB.BINARY, name="p3ijch")

# tickets to be sold at each week
s1ich = model2.addVars(ich, name="s1ich")

# use hindsight demand of week 1
for i,c,h in ich:
    fcst1[i,c,h] = 0
    fcst1[i,c,h] = demand1[c,h]*opt_p1ch[c,h] 

s2ijch = model2.addVars(ijch, name="s2ijch")
s3ijkch = model2.addVars(ijkch, name="s3ijkch")

# number of planes to fly
n = model2.addVar(ub=6, vtype=GRB.INTEGER, name="planes")

# Price option constraints for week 1

priceOption1 = model2.addConstrs( (gp.quicksum(p1ch[c,h] for h in options ) == 1 for c in classes ), name='priceOption1' )

# sales constraints for week 1

sales1 = model2.addConstrs( (s1ich[i,c,h] <= fcst1[i,c,h]*p1ch[c,h] for i,c,h in ich ), name='sales1' )

# Price option constraints for week 2

priceOption2 = model2.addConstrs( (gp.quicksum(p2ich[i,c,h] for h in options ) 
                                  == 1 for i in scenarios for c in classes ), name='priceOption2' )

# sales constraints for week 2

sales2 = model2.addConstrs( (s2ijch[i,j,c,h] <= fcst2[j,c,h]*p2ich[i,c,h] for i,j,c,h in ijch ), name='sales2' )

# Price option constraints for week 3

priceOption3 = model2.addConstrs( (gp.quicksum(p3ijch[i,j,c,h] for h in options ) 
                                  == 1 for i in scenarios for j in scenarios for c in classes ), name='priceOption3' )

# sales constraints for week 3

sales3 = model2.addConstrs( (s3ijkch[i,j,k,c,h] <= fcst3[k,c,h]*p3ijch[i,j,c,h] for i,j,k,c,h in ijkch ), name='sales3' )

# Class capacity constraints.

classCap = model2.addConstrs( (gp.quicksum(s1ich[i,c,h] for h in options)  
                              + gp.quicksum(s2ijch[i,j,c,h] for h in options) 
                              + gp.quicksum(s3ijkch[i,j,k,c,h] for h in options)  
                              <= cap[c]*n for i,j,k,c in ijkc ) , name='classCap')

# Objective function
obj = gp.quicksum(prob[i]*price1[c,h]*p1ch[c,h]*s1ich[i,c,h] for i,c,h in ich ) \
+ gp.quicksum(prob[i]*prob[j]*price2[c,h]*p2ich[i,c,h]*s2ijch[i,j,c,h] for i,j,c,h in ijch ) \
+ gp.quicksum(prob[i]*prob[j]*prob[k]*price3[c,h]*p3ijch[i,j,c,h]*s3ijkch[i,j,k,c,h] for i,j,k,c,h in ijkch) - cost*n

model2.setObjective( obj, GRB.MAXIMIZE)

# Verify model formulation

model2.write('YieldManagement2.lp')

# Run optimization engine

model2.optimize()

#############################################################
#            Print results of model for week 2
#############################################################

print("\n\n\n____________________Week 2 solution___________________________")

print(f"The expected total profit at the beginning of week 2 is: £ {round(model2.objVal,2):,}") 
print(f"Number of planes to book: {n.x}")

# Week 2 prices

# optimal values of option prices at week 1
opt_p2ich = {}

print("\n_____________Week 2 prices____________________________")
for i,c,h in ich:
    opt_p2ich[i,c,h] = 0
    if p2ich[i,c,h].x > 0.5:
        opt_p2ich[i,c,h] = round(p2ich[i,c,h].x)
        price_ch = opt_p2ich[i,c,h]*price2[c,h]
        #print(f"({i},{c},{h}) = £ {price_ch}")
        if i == 'sce1':
            print(f"({c},{h}) = £{price_ch: ,}")
#

# Week 3 provisional prices

print("\n_____________Week 3 provisional prices____________________________")
for i,j,c,h in ijch:
    if p3ijch[i,j,c,h].x > 0.5:
        price_ch = round(p3ijch[i,j,c,h].x)*price3[c,h]
        print(f"({i}, {j}, {c}, {h}) = £ {price_ch}")

Changed value of parameter nonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 471 rows, 469 columns and 1611 nonzeros
Model fingerprint: 0x92c92b4e
Model has 351 quadratic objective terms
Variable types: 351 continuous, 118 integer (117 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [5e+04, 5e+04]
  QObjective range [9e-01, 2e+03]
  Bounds range     [1e+00, 6e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -300000.0000
Presolve removed 30 rows and 27 columns
Presolve time: 0.00s
Presolved: 765 rows, 766 columns, 2376 nonzeros
Variable types: 657 continuous, 109 integer (108 binary)

Root relaxation: objective -1.743903e+05, 456 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incum

# Third Period model
We ran the model using the actual demand of weeks 1 and 2.

In [15]:
model3 = gp.Model('YieldManagement3')

# Set global parameters. 
model3.params.nonConvex = 2

# Decision variables

# price option binary variables at each week
p1ch = model3.addVars(ch, vtype=GRB.BINARY, name="p1ch")

# Fix price options of week 1

for c,h in ch:
    p1ch[c,h].lb = opt_p1ch[c,h]

p2ich = model3.addVars(ich, vtype=GRB.BINARY, name="p2ich")

# Fix price options of week 2

for i,c,h in ich:
    p2ich[i,c,h].lb = opt_p2ich[i,c,h]

p3ijch = model3.addVars(ijch, vtype=GRB.BINARY, name="p3ijch")

# tickets to be sold at each week
s1ich = model3.addVars(ich, name="s1ich")

# use hindsight demand of week 1
for i,c,h in ich:
    fcst1[i,c,h] = 0
    fcst1[i,c,h] = demand1[c,h]*opt_p1ch[c,h]

s2ijch = model3.addVars(ijch, name="s2ijch")

# use hindsight demand of week 2
for j,c,h in ich:
    fcst2[j,c,h] = 0
    fcst2[j,c,h] = demand2[c,h]*opt_p2ich[j,c,h]

s3ijkch = model3.addVars(ijkch, name="s3ijkch")

# number of planes to fly
n = model3.addVar(ub=6, vtype=GRB.INTEGER, name="planes")

# Price option constraints for week 1

priceOption1 = model3.addConstrs( (gp.quicksum(p1ch[c,h] for h in options ) == 1 for c in classes ), name='priceOption1' )

# sales constraints for week 1

sales1 = model3.addConstrs( (s1ich[i,c,h] <= fcst1[i,c,h]*p1ch[c,h] for i,c,h in ich ), name='sales1' )

# Price option constraints for week 2

priceOption2 = model3.addConstrs( (gp.quicksum(p2ich[i,c,h] for h in options ) 
                                  == 1 for i in scenarios for c in classes ), name='priceOption2' )

# sales constraints for week 2

sales2 = model3.addConstrs( (s2ijch[i,j,c,h] <= fcst2[j,c,h]*p2ich[i,c,h] for i,j,c,h in ijch ), name='sales2' )

# Price option constraints for week 3

priceOption3 = model3.addConstrs( (gp.quicksum(p3ijch[i,j,c,h] for h in options ) 
                                  == 1 for i in scenarios for j in scenarios for c in classes ), name='priceOption3' )

# sales constraints for week 3

sales3 = model3.addConstrs( (s3ijkch[i,j,k,c,h] <= fcst3[k,c,h]*p3ijch[i,j,c,h] for i,j,k,c,h in ijkch ), name='sales3' )

# Class capacity constraints.

classCap = model3.addConstrs( (gp.quicksum(s1ich[i,c,h] for h in options)  
                              + gp.quicksum(s2ijch[i,j,c,h] for h in options) 
                              + gp.quicksum(s3ijkch[i,j,k,c,h] for h in options)  
                              <= cap[c]*n for i,j,k,c in ijkc ) , name='classCap')

# Objective function
obj = gp.quicksum(prob[i]*price1[c,h]*p1ch[c,h]*s1ich[i,c,h] for i,c,h in ich ) \
+ gp.quicksum(prob[i]*prob[j]*price2[c,h]*p2ich[i,c,h]*s2ijch[i,j,c,h] for i,j,c,h in ijch ) \
+ gp.quicksum(prob[i]*prob[j]*prob[k]*price3[c,h]*p3ijch[i,j,c,h]*s3ijkch[i,j,k,c,h] for i,j,k,c,h in ijkch) - cost*n

model3.setObjective( obj, GRB.MAXIMIZE)

# Verify model formulation

model3.write('YieldManagement3.lp')

# Run optimization engine

model3.optimize()

#############################################################
#            Print results of model for week 3
#############################################################


print("\n\n\n____________________Week 3 solution___________________________")

print(f"The expected total profit is: £ {round(model3.objVal,2):,}") 
print(f"Number of planes to book: {n.x}")

# Week 3  prices

# optimal values of option prices at week 3
opt_p3ijch = {}


print("\n_____________Week 3 prices____________________________")
for i,j,c,h in ijch:
    opt_p3ijch[i,j,c,h] = 0
    if p3ijch[i,j,c,h].x > 0.5:
        opt_p3ijch[i,j,c,h] = round(p3ijch[i,j,c,h].x)
        price_ch = opt_p3ijch[i,j,c,h]*price3[c,h]
        if i == 'sce1' and j == 'sce1':
            print(f"({c}, {h}) = £{price_ch: ,}")

Changed value of parameter nonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 471 rows, 469 columns and 1557 nonzeros
Model fingerprint: 0x9217fd30
Model has 351 quadratic objective terms
Variable types: 351 continuous, 118 integer (117 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [5e+04, 5e+04]
  QObjective range [9e-01, 2e+03]
  Bounds range     [1e+00, 6e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -300000.0000
Presolve removed 120 rows and 108 columns
Presolve time: 0.00s
Presolved: 594 rows, 604 columns, 1782 nonzeros
Variable types: 522 continuous, 82 integer (81 binary)

Root relaxation: objective -1.786023e+05, 389 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incum

# Solution at take-off
We ran the model using the actual demand of weeks 1, 2, and 3.

In [16]:
model4 = gp.Model('YieldManagement4')

# Set global parameters 
model4.params.nonConvex = 2

# Decision variables

# price option binary variables at each week
p1ch = model4.addVars(ch, vtype=GRB.BINARY, name="p1ch")

# Fix price options of week 1

for c,h in ch:
    p1ch[c,h].lb = opt_p1ch[c,h]

p2ich = model4.addVars(ich, vtype=GRB.BINARY, name="p2ich")

# Fix price options of week 2

for i,c,h in ich:
    p2ich[i,c,h].lb = opt_p2ich[i,c,h]

p3ijch = model4.addVars(ijch, vtype=GRB.BINARY, name="p3ijch")

# Capture one scenario where opt_p3ijch[i,j,c,h] = 1
opt_p3kch = {}

for i,j,c,h in ijch:
    p3ijch[i,j,c,h].lb = opt_p3ijch[i,j,c,h]
    opt_p3kch[j,c,h] = 0
    if opt_p3ijch[i,j,c,h] == 1:
        opt_p3kch[j,c,h] = opt_p3ijch[i,j,c,h] 

# tickets to be sold at each week
s1ich = model4.addVars(ich, name="s1ich")

# use hindsight demand of week 1
for i,c,h in ich:
    fcst1[i,c,h] = 0
    fcst1[i,c,h] = demand1[c,h]*opt_p1ch[c,h] 

s2ijch = model4.addVars(ijch, name="s2ijch")

# use hindsight demand of week 2
for j,c,h in ich:
    fcst2[j,c,h] = 0
    fcst2[j,c,h] = demand2[c,h]*opt_p2ich[j,c,h]


s3ijkch = model4.addVars(ijkch, name="s3ijkch")

# use hindsight demand of week 3
for k,c,h in ich:
    fcst3[k,c,h] = 0
    fcst3[k,c,h] = demand3[c,h]*opt_p3kch[k,c,h]

    
# number of planes to fly
n = model4.addVar(ub=6, vtype=GRB.INTEGER, name="planes")

# Price option constraints for week 1

priceOption1 = model4.addConstrs( (gp.quicksum(p1ch[c,h] for h in options ) == 1 for c in classes ), name='priceOption1' )

# sales constraints for week 1

sales1 = model4.addConstrs( (s1ich[i,c,h] <= fcst1[i,c,h]*p1ch[c,h] for i,c,h in ich ), name='sales1' )

# Price option constraints for week 2

priceOption2 = model4.addConstrs( (gp.quicksum(p2ich[i,c,h] for h in options ) 
                                  == 1 for i in scenarios for c in classes ), name='priceOption2' )

# sales constraints for week 2

sales2 = model4.addConstrs( (s2ijch[i,j,c,h] <= fcst2[j,c,h]*p2ich[i,c,h] for i,j,c,h in ijch ), name='sales2' )

# Price option constraints for week 3

priceOption3 = model4.addConstrs( (gp.quicksum(p3ijch[i,j,c,h] for h in options ) 
                                  == 1 for i in scenarios for j in scenarios for c in classes ), name='priceOption3' )

# sales constraints for week 3

sales3 = model4.addConstrs( (s3ijkch[i,j,k,c,h] <= fcst3[k,c,h]*p3ijch[i,j,c,h] for i,j,k,c,h in ijkch ), name='sales3' )

# Class capacity constraints.

classCap = model4.addConstrs( (gp.quicksum(s1ich[i,c,h] for h in options)  
                              + gp.quicksum(s2ijch[i,j,c,h] for h in options) 
                              + gp.quicksum(s3ijkch[i,j,k,c,h] for h in options)  
                              <= cap[c]*n for i,j,k,c in ijkc ) , name='classCap')

# Objective function
obj = gp.quicksum(prob[i]*price1[c,h]*p1ch[c,h]*s1ich[i,c,h] for i,c,h in ich ) \
+ gp.quicksum(prob[i]*prob[j]*price2[c,h]*p2ich[i,c,h]*s2ijch[i,j,c,h] for i,j,c,h in ijch ) \
+ gp.quicksum(prob[i]*prob[j]*prob[k]*price3[c,h]*p3ijch[i,j,c,h]*s3ijkch[i,j,k,c,h] for i,j,k,c,h in ijkch) - cost*n

model4.setObjective( obj, GRB.MAXIMIZE)

# Verify model formulation

model4.write('YieldManagement4.lp')

# Run optimization engine

model4.optimize()

#############################################################
#            Print results of model for week 4
#############################################################


print("\n\n\n____________________Take off solution___________________________")

print(f"The actual total profit is: £{round(model4.objVal,2):,}") 
print(f"Number of planes used: {n.x}")


# Sales week 1

print("\n___________Week 1 seats sold and revenue__________________________")
for i,c,h in ich:
    if i == 'sce1':
        if s1ich[i,c,h].x > 1e-6:
            tickets = round(s1ich[i,c,h].x)
            price = price1[c,h]*round(p1ch[c,h].x)
            revenue = price*tickets
            print(f"{c} class: {tickets} seats sold at £{price:,}: revenue £{revenue:,}  ")
            
# Sales week 2

print("___________Period 2 seats sold and revenue__________________________")
for i,j,c,h in ijch:
    if i == 'sce1' and j == 'sce1':
        if s2ijch[i,j,c,h].x > 1e-6:
            tickets = round(s2ijch[i,j,c,h].x)
            price = price2[c,h]*round(p2ich[i,c,h].x)
            revenue = price*tickets
            print(f"{c} class: {tickets} seats sold at £{price:,}: revenue £{revenue:,}  ")
            
# Sales week 3

print("___________Period 3 seats sold and revenue__________________________")
for i,j,k,c,h in ijkch:
    if i == 'sce1' and j == 'sce1' and k == 'sce1':
        if s3ijkch[i,j,k,c,h].x > 1e-6:
            tickets = round(s3ijkch[i,j,k,c,h].x )
            price = price3[c,h]*round(p3ijch[i,j,c,h].x)
            revenue = price*tickets
            print(f"{c} class: {tickets} seats sold at £{price:,}: revenue £{revenue:,}  ")

Changed value of parameter nonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 471 rows, 469 columns and 1395 nonzeros
Model fingerprint: 0x0aff6c8c
Model has 351 quadratic objective terms
Variable types: 351 continuous, 118 integer (117 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [5e+04, 5e+04]
  QObjective range [9e-01, 2e+03]
  Bounds range     [1e+00, 6e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 470 rows and 458 columns
Presolve time: 0.00s
Presolved: 1 rows, 11 columns, 11 nonzeros
Variable types: 10 continuous, 1 integer (0 binary)

Root relaxation: objective 1.952617e+05, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestB

## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Copyright © 2020 Gurobi Optimization, LLC